In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
data = Path('combine_data.csv')
df = pd.read_csv(data)

In [4]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

In [5]:
# Drop the null rows
df = df.dropna()

df = df.drop(columns ='name')

In [6]:
df.reset_index(inplace=True, drop=True)

df.head()

,rating,review_count,price,latitude,longitude,stars,michelin,latBin,lonBin
0,4.0,735,4,37.394680,-122.08044,1,1,37.2,-122.2
1,4.5,348,4,37.427853,-122.14362,1,1,37.4,-122.2
2,3.5,886,3,37.420140,-122.21151,1,1,37.4,-122.4
3,4.0,1088,4,37.428970,-122.25178,1,1,37.4,-122.4
4,4.0,1244,4,37.256480,-122.03537,1,1,37.2,-122.2


In [7]:
# Split our preprocessed data into our features and target arrays
y = df['michelin']
X = df.drop(columns='michelin')



In [8]:
X.describe()

,rating,review_count,price,latitude,longitude,stars,latBin,lonBin
count,7110.000000,7110.000000,7110.000000,7110.000000,7110.000000,7110.000000,7110.000000,7110.000000
mean,4.057243,682.871589,2.207454,37.686195,-102.246969,0.032771,37.576259,-102.357665
std,0.449551,952.921199,0.517659,3.091578,20.632329,0.224208,3.085949,20.634838
min,1.000000,1.000000,0.000000,32.691728,-122.886470,0.000000,32.600000,-123.000000
25%,4.000000,167.250000,2.000000,34.088765,-121.919090,0.000000,34.000000,-122.000000
50%,4.000000,382.000000,2.000000,37.781908,-117.160068,0.000000,37.600000,-117.200000
75%,4.500000,822.750000,2.000000,40.710849,-77.045175,0.000000,40.600000,-77.200000
max,5.000000,17070.000000,4.000000,53.512863,-2.946107,3.000000,53.400000,-3.000000


In [9]:
# Check the balance of our target values
y.value_counts()

0    6931
1     179
Name: michelin, dtype: int64

### Naive Random Oversampling

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

In [11]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({0: 5198, 1: 5198})

In [12]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [13]:
# Calcupate predicitons
y_pred = model.predict(X_test)

# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

1.0

In [14]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual no_star", "Actual star"], columns=["Predicted no_star", "Predicted star"])
cm_df

,Predicted no_star,Predicted star
Actual no_star,1733,0
Actual star,0,45


In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00      1733
          1       1.00      1.00      1.00      1.00      1.00      1.00        45

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      1778



### SMOTE Oversampling

In [16]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
# Instantiate the model
smote = SMOTE(random_state=1)
# Resample the targets
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 5198, 1: 5198})

In [17]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [19]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[1733,    0],
       [   0,   45]])

In [20]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00      1733
          1       1.00      1.00      1.00      1.00      1.00      1.00        45

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      1778



# Undersampling

In [21]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 134, 1: 134})

In [22]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=78)

In [23]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

1.0

In [24]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
cm3 = confusion_matrix(y_test, y_pred)
# Create a DataFrame from the confusion matrix.
cm3_df = pd.DataFrame(
    cm3, index=["Actual no_star", "Actual star"], columns=["Predicted no_star", "Predicted star"])
cm3_df

,Predicted no_star,Predicted star
Actual no_star,1676,57
Actual star,0,45


In [25]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.97      1.00      0.98      0.98      0.96      1733
          1       0.44      1.00      0.97      0.61      0.98      0.97        45

avg / total       0.99      0.97      1.00      0.97      0.98      0.96      1778



# Combination (Over and Under) Sampling

In [26]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({0: 5896, 1: 6123})

In [27]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [28]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.9835545297172533

In [29]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)


array([[1733,    0],
       [   0,   45]])

In [30]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00      1733
          1       1.00      1.00      1.00      1.00      1.00      1.00        45

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      1778



### Balanced Random Forest Classifier

In [31]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier(n_estimators = 100)
random_forest = random_forest.fit(X_train, y_train)

In [32]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [33]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1733,    0],
       [   0,   45]])

In [34]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00      1733
          1       1.00      1.00      1.00      1.00      1.00      1.00        45

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      1778



In [35]:
# List the features sorted in descending order by feature importance
feature_names = X.columns
sorted(zip(random_forest.feature_importances_, feature_names), reverse=True)

[(0.562708958506902, 'stars'),
 (0.28100073094631106, 'price'),
 (0.0481119492063125, 'longitude'),
 (0.04212045547382817, 'latitude'),
 (0.024296750453774607, 'review_count'),
 (0.020148220510476243, 'latBin'),
 (0.01141479523678043, 'rating'),
 (0.010198139665614954, 'lonBin')]

### Easy Ensemble AdaBoost Classifier

In [36]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easy = EasyEnsembleClassifier(n_estimators = 100,random_state=1)
easy = easy.fit(X_train, y_train)

In [37]:
# Calculated the balanced accuracy score
y_pred = easy.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [38]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[1733,    0],
       [   0,   45]])

In [39]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00      1733
          1       1.00      1.00      1.00      1.00      1.00      1.00        45

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      1778

